In [1]:
import os
import CityHub
import geopandas as gpd
import pandas as pd
#import numpy as np

#import math
#from statistics import mean

In [2]:
# carrega o citygraph preprocessado
ch=CityHub.load_preprocessed_CityHub('data/sp-6-abr-23.bin')

# Weather Test

In [3]:
# Compute the layer on desired time window
ch.compute_layer_sparse('10-01-2011','11-01-2011')
ch.SMLayers_estimated_measurements

,Precipitacao_Total,Temperatura_Maxima,Temperatura_Minima
corner,,,
0,0.090992,33.841366,16.514428
1,0.089862,33.841250,16.520001
2,0.088726,33.840934,16.535168
3,0.088043,33.841507,16.507651
4,0.079078,33.851935,16.007117
...,...,...,...
121537,0.148769,33.814809,17.789174
121538,0.145225,33.814890,17.785263
121539,0.145882,33.814702,17.794285


# Crime Test

In [4]:
from shapely.geometry import Polygon, Point

city_vert_gdf = gpd.GeoDataFrame(list(range(0,len(ch.city_vert_list))),geometry = gpd.points_from_xy(pd.DataFrame(ch.city_vert_list)[0], pd.DataFrame(ch.city_vert_list)[1]))
city_vert_gdf.columns = ["vert", "geometry"]
city_vert_gdf

,vert,geometry
0,0,POINT (-23.56299 -46.70397)
1,1,POINT (-23.56746 -46.70223)
2,2,POINT (-23.57296 -46.69923)
3,3,POINT (-23.57183 -46.70207)
4,4,POINT (-23.53462 -46.74815)
...,...,...
121537,121537,POINT (-23.55463 -46.53176)
121538,121538,POINT (-23.57349 -46.49451)
121539,121539,POINT (-23.57281 -46.49286)
121540,121540,POINT (-23.56551 -46.48308)


In [8]:
import types

def compute_layer_crimes(layer, ch, initial_date = None, final_date = None):

        tmp_layer_crimes = ch.PBLayers[layer].copy()
        if((not initial_date is None) or (not final_date is None)):
            tmp_layer_crimes["time"] = pd.to_datetime(tmp_layer_crimes["time"], format = "%Y/%m/%d", errors = 'coerce')
            tmp_layer_crimes = tmp_layer_crimes[tmp_layer_crimes['time'].notna()]
            if(not initial_date is None):
                tmp_layer_crimes = tmp_layer_crimes[tmp_layer_crimes["time"] >= initial_date]
            if(not final_date is None):
                tmp_layer_crimes = tmp_layer_crimes[tmp_layer_crimes["time"] <= final_date]


        aux = pd.DataFrame(tmp_layer_crimes[["vert"]].value_counts().keys().tolist(), columns = ["vert"])

        aux["n"] = tmp_layer_crimes[["vert"]].value_counts().values
        #print(aux)
        #print(tmp_layer_crimes)
        
        df_vert = pd.DataFrame()
        df_vert['vert'] = list(ch.city_vert_dict.values())
        
        df_vert = df_vert.merge(aux, how='left', on='vert')
        df_vert['n'] = df_vert['n'].fillna(0)

        return df_vert, tmp_layer_crimes

In [13]:
import datetime

#Calcular X das features dinâmicas
x1 = datetime.datetime(2011, 1, 10)
x2 = datetime.datetime(2011, 1, 11)#não inclui dia 11 (aparentemente), 00:00:00 conta o dia anterior
X_dynamic, crimes_vert = compute_layer_crimes(layer = 4, ch = ch, initial_date = x1, final_date = x2)
X_dynamic

,vert,n
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
121537,121537,0.0
121538,121538,0.0
121539,121539,0.0
121540,121540,0.0


In [14]:
crimes_vert

,Unnamed: 0,latitude,longitude,time,latlong,vert
244888,244888,"-23,559607","-46,412452",2011-01-10 00:30:00,"(-23.559607, -46.412452)",64261
244889,244889,"-23,610889","-46,450152",2011-01-10 00:30:00,"(-23.610889, -46.450152)",87197
244890,244890,"-23,467219","-46,592479",2011-01-10 00:30:00,"(-23.467219, -46.592479)",46776
244891,244891,"-23,544872","-46,643439",2011-01-10 00:30:00,"(-23.544872, -46.643439)",89815
244892,244892,"-23,658887","-46,759794",2011-01-10 00:35:00,"(-23.658887, -46.759794)",71250
...,...,...,...,...,...,...
245031,245031,"-23,461928","-46,594604",2011-01-10 23:30:00,"(-23.461928, -46.594604)",68484
245032,245032,"-23,608255","-46,465920",2011-01-10 23:30:00,"(-23.608255, -46.46592)",43799
245033,245033,"-23,620192","-46,693361",2011-01-10 23:40:00,"(-23.620192, -46.693361)",90636
245034,245034,"-23,601131","-46,747999",2011-01-10 23:45:00,"(-23.601131, -46.747999)",116634


In [11]:
len(X_dynamic[X_dynamic['n'] != 0])

145

In [12]:
X_dynamic.n.sum()

148.0

In [29]:
X_dynamic.n.max()

2.0

### Análise de crimes e nós para o ano de 2011

In [57]:
dates = list(pd.date_range('2011-01-01','2012-01-01'))
X_crimes = pd.DataFrame(columns=['date', 'nodes', 'qnt_crimes'])

for i in range(0, len(dates)-1):
    d1 = dates[i]
    d2 = dates[i+1]
    X_dynamic = compute_layer_crimes(layer = 4, ch = ch, initial_date = d1, final_date = d2)
    nodes_with_crimes = len(X_dynamic[X_dynamic['n'] != 0])
    qnt_crimes = X_dynamic.n.sum()
    X_crimes.loc[len(X_crimes.index)] = [d1, nodes_with_crimes, qnt_crimes] 

In [58]:
X_crimes

,date,nodes,qnt_crimes
0,2011-01-01,103,107.0
1,2011-01-02,131,134.0
2,2011-01-03,179,183.0
3,2011-01-04,145,146.0
4,2011-01-05,189,193.0
...,...,...,...
360,2011-12-27,152,154.0
361,2011-12-28,138,139.0
362,2011-12-29,137,138.0
363,2011-12-30,110,111.0


In [62]:
X_crimes.mean()

/tmp/ipykernel_266210/1508018025.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  X_crimes.mean()


nodes         165.986301
qnt_crimes    169.797260
dtype: float64

In [67]:
(X_crimes.mean()[0]/len(city_vert_gdf))*100

/tmp/ipykernel_266210/4262022121.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  (X_crimes.mean()[0]/len(city_vert_gdf))*100


0.13656703145403482